In [ ]:
#install pyspark library 
!pip install pyspark

In [1]:
#import pyspark
import pyspark

In [2]:
#import sparksession
from pyspark.sql import SparkSession

In [ ]:
#creating a sparksession object providing appName
spark=SparkSession.builder.appName('optimization').getOrCreate()

In [ ]:
#To create dataframe form External datasets
AirlineDF = spark.read.option("header","true").csv(r"C:\\Users\\User\\Downloads\\partitions\\Data\\*")

<h1> use take() in place of collect() for reduce time </h1>

In [ ]:
%time AirlineDF.show(5)

In [ ]:
#calculate rows in dataframe
%time AirlineDF.count()

In [ ]:
#calculate total columns in dataframe
len(AirlineDF.columns)

In [ ]:
%time AirlineDF.take(5)

<h1> Catalyst optimizer </h1>
 Catalyst Optimizer improves developer productivity and the performance of their written queries.

In [ ]:
%time AirlineDF.select('OriginCityname') \
.filter("Distance >= 200").filter("ArrDelayMinutes >= 5").filter("Year >= 2005").groupBy('OriginCityName') \
.count().withColumnRenamed("count", "Total Delay Flights").orderBy("OriginCityName").show()

In [ ]:
%time AirlineDF.select('OriginCityname') \
.filter("Distance >= 200").filter("ArrDelayMinutes >= 5").filter("Year >= 2005").groupBy('OriginCityName') \
.count().withColumnRenamed("count", "Total Delay Flights").orderBy("OriginCityName").explain(mode="formatted")

In [ ]:
%time AirlineDF.select('OriginCityname') \
.filter("Distance >= 200").filter("ArrDelayMinutes >= 5").filter("Year >= 2005").groupBy('OriginCityName') \
.count().withColumnRenamed("count", "Total Delay Flights").orderBy("OriginCityName").explain(True)

In [ ]:
from pyspark.sql.functions import col

In [ ]:
%time AirlineDF.select('OriginCityName') \
.filter((col('Distance') >= 200 ) & (col('ArrDelayMinutes') >= 5) & (col("Year") >= 2005))\
.groupBy('OriginCityName') \
.count().orderBy('OriginCityName') \
.withColumnRenamed("count", "Total Delay Flights").show()

In [ ]:
AirlineDF.select('OriginCityName') \
.filter((col('Distance') >= 200 ) & (col('ArrDelayMinutes') >= 5) & (col("Year") >= 2005))\
.groupBy('OriginCityName') \
.count().orderBy('OriginCityName') \
.withColumnRenamed("count", "Total Delay Flights").explain(mode = "formatted")

explain(mode="simple") shows physical plan 
explain(mode="extended") present physical and logical plan 
explain(mode="codegen") shows the java code planned to be executed
explain(mode="cost") presents the optimized logical plan and related statistics
explain(mode="formatted")shows a split output created by optimized physical plan outline and a section of node detail

<h1> When data is huge otherwise not </h1>
<h1> use coalesce() in place of repartition() to reduce the no. of partition </h1>

In [ ]:
#To check how many partition in current AirlineDF Dataframe
AirlineDF.rdd.getNumPartitions()

In [ ]:
AirlineDF1 = AirlineDF.repartition(2)

In [ ]:
AirlineDF1.rdd.getNumPartitions()

In [ ]:
AirlineDF2 = AirlineDF.coalesce(2)

In [ ]:
AirlineDF2.rdd.getNumPartitions()

<h1> when we load data from other source like s3, databse </h1>
<h1> when you dealing heavy-weighted initialization on larger datasets. </h1>

<h1> select needeed columns from entire dataset and write into new file then create new dataframe </h1>

In [ ]:
sc.stop()

In [3]:
from pyspark import SparkContext
#create an RDD
sc = SparkContext()

In [4]:
df = sc.textFile(r"C:\\Users\\User\\Downloads\\partitions\\Data\\excel\\*")

In [5]:
Ardd = df.map(lambda x: x.replace(',',' ').split(' '))

In [6]:
AirlineRdd = Ardd.map(lambda x: Row(Year=x[0], UniqueCarrier=x[6]))

In [7]:
from pyspark import SQLContext
from pyspark.sql import Row

In [8]:
#setting up the sql context
sqlContex = SQLContext(sc)

C:\Users\User\anaconda3\lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [9]:
spark = SparkSession.builder.getOrCreate() 

In [10]:
AirlineDf1 = spark.createDataFrame(AirlineRdd).toDF('Year', 'UniqueCarrier')


In [11]:
AirlineDf1 = sqlContex.createDataFrame(AirlineRdd)

In [12]:
print(AirlineRdd)

PythonRDD[12] at RDD at PythonRDD.scala:53


In [13]:
AirlineRdd.collect()

[Row(Year='"Year"', UniqueCarrier='"UniqueCarrier"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Ro

In [14]:
df.take(10)

['"Year","Quarter","Month","DayofMonth","DayOfWeek","FlightDate","UniqueCarrier","AirlineID","Carrier","TailNum","FlightNum","OriginAirportID","OriginAirportSeqID","OriginCityMarketID","Origin","OriginCityName","OriginState","OriginStateFips","OriginStateName","OriginWac","DestAirportID","DestAirportSeqID","DestCityMarketID","Dest","DestCityName","DestState","DestStateFips","DestStateName","DestWac","CRSDepTime","DepTime","DepDelay","DepDelayMinutes","DepDel15","DepartureDelayGroups","DepTimeBlk","TaxiOut","WheelsOff","WheelsOn","TaxiIn","CRSArrTime","ArrTime","ArrDelay","ArrDelayMinutes","ArrDel15","ArrivalDelayGroups","ArrTimeBlk","Cancelled","CancellationCode","Diverted","CRSElapsedTime","ActualElapsedTime","AirTime","Flights","Distance","DistanceGroup","CarrierDelay","WeatherDelay","NASDelay","SecurityDelay","LateAircraftDelay","FirstDepTime","TotalAddGTime","LongestAddGTime","DivAirportLandings","DivReachedDest","DivActualElapsedTime","DivArrDelay","DivDistance","Div1Airport","Div

In [15]:
Ardd.take(10)

[['"Year"',
  '"Quarter"',
  '"Month"',
  '"DayofMonth"',
  '"DayOfWeek"',
  '"FlightDate"',
  '"UniqueCarrier"',
  '"AirlineID"',
  '"Carrier"',
  '"TailNum"',
  '"FlightNum"',
  '"OriginAirportID"',
  '"OriginAirportSeqID"',
  '"OriginCityMarketID"',
  '"Origin"',
  '"OriginCityName"',
  '"OriginState"',
  '"OriginStateFips"',
  '"OriginStateName"',
  '"OriginWac"',
  '"DestAirportID"',
  '"DestAirportSeqID"',
  '"DestCityMarketID"',
  '"Dest"',
  '"DestCityName"',
  '"DestState"',
  '"DestStateFips"',
  '"DestStateName"',
  '"DestWac"',
  '"CRSDepTime"',
  '"DepTime"',
  '"DepDelay"',
  '"DepDelayMinutes"',
  '"DepDel15"',
  '"DepartureDelayGroups"',
  '"DepTimeBlk"',
  '"TaxiOut"',
  '"WheelsOff"',
  '"WheelsOn"',
  '"TaxiIn"',
  '"CRSArrTime"',
  '"ArrTime"',
  '"ArrDelay"',
  '"ArrDelayMinutes"',
  '"ArrDel15"',
  '"ArrivalDelayGroups"',
  '"ArrTimeBlk"',
  '"Cancelled"',
  '"CancellationCode"',
  '"Diverted"',
  '"CRSElapsedTime"',
  '"ActualElapsedTime"',
  '"AirTime"',
  '"Fli

In [16]:
AirlineRdd.take(500)

[Row(Year='"Year"', UniqueCarrier='"UniqueCarrier"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Row(Year='2005', UniqueCarrier='"AA"'),
 Ro

In [17]:
%time AirlineDf1.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

+---------------+------+
|  UniqueCarrier| count|
+---------------+------+
|           "AS"| 26057|
|           "B6"| 19504|
|"UniqueCarrier"|     2|
|           "AA"|112307|
|           "UA"| 81873|
|           "DL"|102709|
|           "DH"| 16279|
|           "CO"| 47567|
|           "US"| 82051|
|           "WN"|176248|
|           "NW"| 74204|
|           "OH"| 57289|
|           "OO"| 84556|
|           "TZ"|  7174|
|           "XE"| 63619|
|           "FL"| 33319|
|           "EV"| 47102|
|           "HA"|  8068|
|           "HP"| 16255|
|           "MQ"| 88006|
+---------------+------+
only showing top 20 rows

Wall time: 19.3 s


In [18]:
AirlineDf1.write.option("header","true").csv("flight/airdata")

AnalysisException: path file:/C:/Users/User/flight/airdata already exists.

In [19]:
#import sparksession
from pyspark.sql import SparkSession

In [20]:
#creating a sparksession object and providing appname
spark=SparkSession.builder.appName("optimization").getOrCreate()

In [21]:
#To create dataframe form External datasets 
AirlineDf2 = spark.read.option("header","true").csv("flight/airdata/*")

In [22]:
AirlineDf2.show()

+----+-------------+
|Year|UniqueCarrier|
+----+-------------+
|2005|         "WN"|
|2005|         "WN"|
|2005|         "WN"|
|2005|         "WN"|
|2005|         "WN"|
|2005|         "WN"|
|2005|         "WN"|
|2005|         "WN"|
|2005|         "WN"|
|2005|         "WN"|
|2005|         "WN"|
|2005|         "WN"|
|2005|         "WN"|
|2005|         "WN"|
|2005|         "WN"|
|2005|         "WN"|
|2005|         "WN"|
|2005|         "WN"|
|2005|         "WN"|
|2005|         "WN"|
+----+-------------+
only showing top 20 rows



In [23]:
%time AirlineDf2.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

+-------------+------+
|UniqueCarrier| count|
+-------------+------+
|         "FL"| 33319|
|         "EV"| 47102|
|         "HA"|  8068|
|         "DL"|102709|
|         "WN"|176248|
|         "HP"| 16255|
|         "NW"| 74204|
|         "US"| 82051|
|         "YV"| 25132|
|         "UA"| 81873|
|         "XE"| 63619|
|         "B6"| 19504|
|         "DH"| 16279|
|         "CO"| 47567|
|         "OO"| 84556|
|         "TZ"|  7174|
|         "F9"|  6892|
|         "MQ"| 88006|
|         "OH"| 57289|
|         "AS"| 26057|
+-------------+------+
only showing top 20 rows

Wall time: 1.44 s


<h1> Apache Parquet is columnar file format that provides optimizations to speed up queeries and is a far more efficient file format than CSV or JSON, supported by many data processing systems </h1> 

In [24]:
AirlineDf2.write.parquet("parquet/")

AnalysisException: path file:/C:/Users/User/parquet already exists.

In [26]:
NewDfParquet = spark.read.parquet("parquet/*")

In [27]:
%time NewDfParquet.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

+---------------+------+
|  UniqueCarrier| count|
+---------------+------+
|           "OH"| 57289|
|           "AS"| 26057|
|           "B6"| 19504|
|"UniqueCarrier"|     2|
|           "AA"|112307|
|           "MQ"| 88006|
|           "OO"| 84556|
|           "NW"| 74204|
|           "UA"| 81873|
|           "US"| 82051|
|           "WN"|176248|
|           "FL"| 33319|
|           "EV"| 47102|
|           "TZ"|  7174|
|           "XE"| 63619|
|           "HA"|  8068|
|           "DL"|102709|
|           "F9"|  6892|
|           "DH"| 16279|
|           "CO"| 47567|
+---------------+------+
only showing top 20 rows

Wall time: 1.15 s


<h1> Using cache() and persist() methods, Spark provides an optimization mechanism to store the intermediate computation of a Spark DataFrame so they can be reused in subsequent actions. </h1>

<h1> NOTE - When NOT to use Cache/Persist- When the size of the data is large and there are multiple dfs available for cache. </h1>

In [28]:
# before rdd is persisted
%time AirlineDf1.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

+---------------+------+
|  UniqueCarrier| count|
+---------------+------+
|           "AS"| 26057|
|           "B6"| 19504|
|"UniqueCarrier"|     2|
|           "AA"|112307|
|           "UA"| 81873|
|           "DL"|102709|
|           "DH"| 16279|
|           "CO"| 47567|
|           "US"| 82051|
|           "WN"|176248|
|           "NW"| 74204|
|           "OH"| 57289|
|           "OO"| 84556|
|           "TZ"|  7174|
|           "XE"| 63619|
|           "FL"| 33319|
|           "EV"| 47102|
|           "HA"|  8068|
|           "HP"| 16255|
|           "MQ"| 88006|
+---------------+------+
only showing top 20 rows

Wall time: 16.8 s


In [29]:
%time AirlineDf1.select("Year").groupby("Year").count().show()

+------+------+
|  Year| count|
+------+------+
|  2005|594924|
|"Year"|     2|
|  2006|581287|
+------+------+

Wall time: 18.2 s


In [30]:
from pyspark import StorageLevel

In [31]:
#persist dataframe to memory disk
AirlineDf1.persist(StorageLevel.MEMORY_AND_DISK)

DataFrame[Year: string, UniqueCarrier: string]

In [32]:
%time AirlineDf1.select("Year").groupby("Year").count().show()

+------+------+
|  Year| count|
+------+------+
|  2005|594924|
|"Year"|     2|
|  2006|581287|
+------+------+

Wall time: 15.6 s


In [33]:
%time AirlineDf1.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

+---------------+------+
|  UniqueCarrier| count|
+---------------+------+
|           "AS"| 26057|
|           "B6"| 19504|
|"UniqueCarrier"|     2|
|           "AA"|112307|
|           "UA"| 81873|
|           "DL"|102709|
|           "DH"| 16279|
|           "CO"| 47567|
|           "US"| 82051|
|           "WN"|176248|
|           "NW"| 74204|
|           "OH"| 57289|
|           "OO"| 84556|
|           "TZ"|  7174|
|           "XE"| 63619|
|           "FL"| 33319|
|           "EV"| 47102|
|           "HA"|  8068|
|           "HP"| 16255|
|           "MQ"| 88006|
+---------------+------+
only showing top 20 rows

Wall time: 376 ms


In [34]:
#unpersist dataframe 
AirlineDf1.unpersist()

DataFrame[Year: string, UniqueCarrier: string]

In [35]:
#persist dataframe to memory
AirlineDf1.persist(StorageLevel.DISK_ONLY)

DataFrame[Year: string, UniqueCarrier: string]

In [36]:
%time AirlineDf1.select("Year").groupby("Year").count().show()

+------+------+
|  Year| count|
+------+------+
|  2005|594924|
|"Year"|     2|
|  2006|581287|
+------+------+

Wall time: 14.6 s


In [37]:
%time AirlineDf1.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

+---------------+------+
|  UniqueCarrier| count|
+---------------+------+
|           "AS"| 26057|
|           "B6"| 19504|
|"UniqueCarrier"|     2|
|           "AA"|112307|
|           "UA"| 81873|
|           "DL"|102709|
|           "DH"| 16279|
|           "CO"| 47567|
|           "US"| 82051|
|           "WN"|176248|
|           "NW"| 74204|
|           "OH"| 57289|
|           "OO"| 84556|
|           "TZ"|  7174|
|           "XE"| 63619|
|           "FL"| 33319|
|           "EV"| 47102|
|           "HA"|  8068|
|           "HP"| 16255|
|           "MQ"| 88006|
+---------------+------+
only showing top 20 rows

Wall time: 497 ms


In [38]:
#unpersist dataframe
AirlineDf1.unpersist()

DataFrame[Year: string, UniqueCarrier: string]

In [39]:
#cache dataframe 
AirlineDf1.cache()

DataFrame[Year: string, UniqueCarrier: string]

In [40]:
%time AirlineDf1.select("Year").groupby("Year").count().show()

+------+------+
|  Year| count|
+------+------+
|  2005|594924|
|"Year"|     2|
|  2006|581287|
+------+------+

Wall time: 15 s


In [41]:
%time AirlineDf1.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

+---------------+------+
|  UniqueCarrier| count|
+---------------+------+
|           "AS"| 26057|
|           "B6"| 19504|
|"UniqueCarrier"|     2|
|           "AA"|112307|
|           "UA"| 81873|
|           "DL"|102709|
|           "DH"| 16279|
|           "CO"| 47567|
|           "US"| 82051|
|           "WN"|176248|
|           "NW"| 74204|
|           "OH"| 57289|
|           "OO"| 84556|
|           "TZ"|  7174|
|           "XE"| 63619|
|           "FL"| 33319|
|           "EV"| 47102|
|           "HA"|  8068|
|           "HP"| 16255|
|           "MQ"| 88006|
+---------------+------+
only showing top 20 rows

Wall time: 369 ms


In [42]:
#unpersist dataframe 
AirlineDf1.unpersist()

DataFrame[Year: string, UniqueCarrier: string]

In [43]:
#cache dataframe
AirlineDf1.cache()

DataFrame[Year: string, UniqueCarrier: string]

In [44]:
%time AirlineDf1.select("Year").groupby("Year").count().show()

+------+------+
|  Year| count|
+------+------+
|  2005|594924|
|"Year"|     2|
|  2006|581287|
+------+------+

Wall time: 16.8 s


In [45]:
#uncache dataframe
AirlineDf1.unpersist()

DataFrame[Year: string, UniqueCarrier: string]

In [46]:
#after rdd is uncached
%time AirlineDf1.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

+---------------+------+
|  UniqueCarrier| count|
+---------------+------+
|           "AS"| 26057|
|           "B6"| 19504|
|"UniqueCarrier"|     2|
|           "AA"|112307|
|           "UA"| 81873|
|           "DL"|102709|
|           "DH"| 16279|
|           "CO"| 47567|
|           "US"| 82051|
|           "WN"|176248|
|           "NW"| 74204|
|           "OH"| 57289|
|           "OO"| 84556|
|           "TZ"|  7174|
|           "XE"| 63619|
|           "FL"| 33319|
|           "EV"| 47102|
|           "HA"|  8068|
|           "HP"| 16255|
|           "MQ"| 88006|
+---------------+------+
only showing top 20 rows

Wall time: 15.5 s


<h1> use reduceByKey in place of Groupbykey </h1>

In [47]:
rdd1 =  sc.textFile(r"C:\\Users\\User\\Downloads\\partitions\\Data\\excel\\*")

In [48]:
#split words using flatMap
rdd_word = rdd1.flatMap(lambda x: x.split(","))

In [49]:
#create a paire-rdd
rdd_pair = rdd_word.map(lambda x: (x, 1))

In [53]:
#Count occurence per word using groupbykey()
rdd_group = rdd_pair.groupByKey()
rdd_group_count = rdd_group.map(lambda x:(x[0], len(x[1])))
%time rdd_group_count.collect()

Wall time: 1min


[('"OriginCityName"', 2),
 ('"Cancelled"', 2),
 ('"CRSElapsedTime"', 2),
 ('"ActualElapsedTime"', 2),
 ('"DivDistance"', 2),
 ('"Div4TotalGTime"', 2),
 ('', 37977595),
 ('1247801', 32668),
 ('32575', 118860),
 ('"California"', 278024),
 ('"0900-0959"', 143576),
 ('27.00', 40764),
 ('10', 66077),
 ('15.00', 142386),
 ('4', 388913),
 ('"0856"', 5030),
 ('430.00', 1063),
 ('-6.00', 73730),
 ('"N330AA"', 124),
 ('10.00', 238429),
 ('380.00', 859),
 ('21.00', 61560),
 ('2005-01-11', 19513),
 ('12', 57654),
 ('"0904"', 5181),
 ('53.00', 32154),
 ('"1137"', 5990),
 ('362.00', 2488),
 ('24', 38499),
 ('2005-01-24', 19769),
 ('-14.00', 22188),
 ('"1147"', 6107),
 ('"2"', 578),
 ('"1017"', 5890),
 ('330.00', 2166),
 ('281.00', 3674),
 ('"1229"', 6159),
 ('394.00', 270),
 ('"1440"', 13546),
 ('"1453"', 5811),
 ('-35.00', 932),
 ('"2000-2059"', 124351),
 ('269.00', 2947),
 ('"1248"', 6606),
 ('"2150"', 7724),
 ('296.00', 5130),
 ('"2103"', 4884),
 ('"1005"', 12500),
 ('"N343AA"', 42),
 ('523.00', 

In [54]:
#count occurrence per word using reducebykey()
rdd_reduce = rdd_pair.reduceByKey(lambda x, y: x+y)
%time rdd_reduce.collect()

Wall time: 1min 2s


[('"OriginCityName"', 2),
 ('"Cancelled"', 2),
 ('"CRSElapsedTime"', 2),
 ('"ActualElapsedTime"', 2),
 ('"DivDistance"', 2),
 ('"Div4TotalGTime"', 2),
 ('', 37977595),
 ('1247801', 32668),
 ('32575', 118860),
 ('"California"', 278024),
 ('"0900-0959"', 143576),
 ('27.00', 40764),
 ('10', 66077),
 ('15.00', 142386),
 ('4', 388913),
 ('"0856"', 5030),
 ('430.00', 1063),
 ('-6.00', 73730),
 ('"N330AA"', 124),
 ('10.00', 238429),
 ('380.00', 859),
 ('21.00', 61560),
 ('2005-01-11', 19513),
 ('12', 57654),
 ('"0904"', 5181),
 ('53.00', 32154),
 ('"1137"', 5990),
 ('362.00', 2488),
 ('24', 38499),
 ('2005-01-24', 19769),
 ('-14.00', 22188),
 ('"1147"', 6107),
 ('"2"', 578),
 ('"1017"', 5890),
 ('330.00', 2166),
 ('281.00', 3674),
 ('"1229"', 6159),
 ('394.00', 270),
 ('"1440"', 13546),
 ('"1453"', 5811),
 ('-35.00', 932),
 ('"2000-2059"', 124351),
 ('269.00', 2947),
 ('"1248"', 6606),
 ('"2150"', 7724),
 ('296.00', 5130),
 ('"2103"', 4884),
 ('"1005"', 12500),
 ('"N343AA"', 42),
 ('523.00', 

<h1> use apache arrow to optimize query time </h1>

<h1> create dataframe in pyspark from pandas </h1> 

In [55]:
!pip install pyarrow

<h1> here we are creating pandas dataframe from multiple csv file so we need to import glob </h1>

In [56]:
import pandas as pd
import numpy as np

In [65]:
pdf1 = pd.read_csv(r"C:\Users\User\Downloads\partitions\Data\excel\On_Time_On_Time_Performance_2005_1.csv")

<h1> how to enable arrow for fast pyspark dataframe creation from pandas dataframe </h1>

In [66]:
spark.conf.set("spark.sql.execution.arrow.enabled","true")

In [68]:
%time df2 = spark.createDataFrame(pdf1)

Wall time: 1min 13s


<h1> how to disable arrow </h1>

In [69]:
spark.conf.set("spark.sql.execution.arrow.enabled","false")

In [1]:
#we need to cast all the columns in the pandas df to string type to overcome this datatype issue converting pandas df 
%time df1 = spark.createDataFrame(pdf1.astype(str))

NameError: name 'spark' is not defined